# Claim 2: SQSTM1 (p62) Upregulation Analysis with PertPy
## Testing: "SQSTM1 is massively upregulated (log2FC = 3.413, FDR = 1.76 × 10^-8)"

This notebook uses PertPy to analyze SQSTM1/p62 expression and its relationship with disease progression (pseudotime).

In [ ]:
import pertpy as pt
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

## 1. Load Data and Find SQSTM1

In [ ]:
# Load prepared data
adata = sc.read_h5ad('../01_data_preparation/prepared_for_pertpy.h5ad')

print(f"Data shape: {adata.shape}")
print(f"Tau-positive: {adata.obs['tau_positive'].sum()}")
print(f"Tau-negative: {(adata.obs['tau_positive'] == 0).sum()}")

# Find SQSTM1
protein_names = adata.var['protein_name'] if 'protein_name' in adata.var else adata.var.index
sqstm1_matches = [p for p in protein_names if 'SQSTM1' in p.upper() or 'P62' in p.upper()]

if sqstm1_matches:
    sqstm1_name = sqstm1_matches[0]
    print(f"\n✓ Found SQSTM1: {sqstm1_name}")
    sqstm1_idx = list(protein_names).index(sqstm1_name)
else:
    print("\n⚠ SQSTM1 not found by exact match, searching for similar names...")
    # Try partial matching
    possible = [p for p in protein_names if 'SQ' in p.upper() or '62' in p]
    if possible:
        print(f"Possible matches: {possible[:5]}")
        sqstm1_name = possible[0]
        sqstm1_idx = list(protein_names).index(sqstm1_name)
    else:
        print("ERROR: SQSTM1 not found in dataset")
        sqstm1_idx = None

## 2. Extract SQSTM1 Expression Data

In [ ]:
if sqstm1_idx is not None:
    # Extract SQSTM1 expression
    sqstm1_expr = adata.X[:, sqstm1_idx]
    
    # Create DataFrame for analysis
    sqstm1_df = pd.DataFrame({
        'expression': sqstm1_expr,
        'tau_status': adata.obs['tau_status'].values,
        'tau_positive': adata.obs['tau_positive'].values,
        'pseudotime': adata.obs['pseudotime'].values if 'pseudotime' in adata.obs else None,
        'mc1_score': adata.obs['mc1_score'].values if 'mc1_score' in adata.obs else None
    })
    
    print("SQSTM1 Expression Statistics:")
    print(sqstm1_df.groupby('tau_status')['expression'].describe())
else:
    print("Cannot proceed without SQSTM1 data")

## 3. Single Protein DGE with PyDESeq2

In [ ]:
if sqstm1_idx is not None:
    # Create single-protein AnnData for SQSTM1
    adata_sqstm1 = adata[:, sqstm1_idx:sqstm1_idx+1].copy()
    
    print(f"SQSTM1 subset shape: {adata_sqstm1.shape}")
    
    # Run PyDESeq2 for SQSTM1
    try:
        # Use counts if available
        if 'counts' in adata_sqstm1.layers:
            adata_sqstm1.layers['log2'] = adata_sqstm1.X.copy()
            adata_sqstm1.X = adata_sqstm1.layers['counts'].copy()
        
        # Initialize PyDESeq2 with pseudotime as covariate if available
        if 'pseudotime' in adata_sqstm1.obs and not adata_sqstm1.obs['pseudotime'].isna().all():
            design = "~tau_status + pseudotime"
            print("Using design with pseudotime covariate")
        else:
            design = "~tau_status"
            print("Using simple tau status design")
        
        pds2 = pt.tl.PyDESeq2(
            adata=adata_sqstm1,
            design=design,
            refit_cooks=True
        )
        
        pds2.fit()
        
        # Test contrast
        results_sqstm1 = pds2.test_contrasts(
            pds2.contrast(
                column="tau_status",
                baseline="negative",
                group_to_compare="positive"
            )
        )
        
        print("\n✓ PyDESeq2 analysis completed")
        print(results_sqstm1)
        
    except Exception as e:
        print(f"PyDESeq2 failed: {e}")
        print("\nUsing traditional statistics...")
        
        # Fallback analysis
        tau_pos = sqstm1_df['tau_positive'] == 1
        tau_neg = sqstm1_df['tau_positive'] == 0
        
        # Calculate statistics
        mean_pos = sqstm1_df.loc[tau_pos, 'expression'].mean()
        mean_neg = sqstm1_df.loc[tau_neg, 'expression'].mean()
        log2fc = mean_pos - mean_neg  # Already log2 transformed
        
        # T-test
        tstat, pval = stats.ttest_ind(
            sqstm1_df.loc[tau_pos, 'expression'],
            sqstm1_df.loc[tau_neg, 'expression']
        )
        
        # Mann-Whitney U test (non-parametric)
        ustat, pval_mw = stats.mannwhitneyu(
            sqstm1_df.loc[tau_pos, 'expression'],
            sqstm1_df.loc[tau_neg, 'expression'],
            alternative='two-sided'
        )
        
        # Cohen's d effect size
        std_pooled = np.sqrt(
            (sqstm1_df.loc[tau_pos, 'expression'].var() * (tau_pos.sum() - 1) +
             sqstm1_df.loc[tau_neg, 'expression'].var() * (tau_neg.sum() - 1)) /
            (tau_pos.sum() + tau_neg.sum() - 2)
        )
        cohen_d = log2fc / std_pooled if std_pooled > 0 else 0
        
        results_sqstm1 = pd.DataFrame([{
            'protein': 'SQSTM1',
            'log2FoldChange': log2fc,
            'pvalue': pval,
            'pvalue_mw': pval_mw,
            'cohen_d': cohen_d,
            'mean_tau_pos': mean_pos,
            'mean_tau_neg': mean_neg
        }])

## 4. Pseudotime Regression Analysis

In [ ]:
if sqstm1_idx is not None and 'pseudotime' in sqstm1_df.columns:
    # Remove NaN values
    valid_mask = ~(sqstm1_df['pseudotime'].isna() | sqstm1_df['expression'].isna())
    
    if valid_mask.sum() > 10:  # Need enough points for regression
        X = sqstm1_df.loc[valid_mask, 'pseudotime'].values.reshape(-1, 1)
        y = sqstm1_df.loc[valid_mask, 'expression'].values
        
        # Linear regression
        lr = LinearRegression()
        lr.fit(X, y)
        
        # Get statistics
        beta = lr.coef_[0]
        y_pred = lr.predict(X)
        
        # Calculate R-squared
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        # Pearson correlation
        corr, corr_pval = stats.pearsonr(X.flatten(), y)
        
        print("\nPseudotime Regression Analysis:")
        print(f"Beta coefficient: {beta:.3f}")
        print(f"R-squared: {r_squared:.3f}")
        print(f"Correlation: {corr:.3f} (p = {corr_pval:.3e})")
        
        # Claimed values
        claimed_beta = 4.951
        print(f"\nClaimed beta: {claimed_beta}")
        print(f"Observed beta: {beta:.3f}")
        print(f"Difference: {abs(beta - claimed_beta):.3f}")
    else:
        print("Insufficient data for pseudotime regression")
        beta = None
        r_squared = None
else:
    print("Pseudotime data not available")
    beta = None
    r_squared = None

## 5. Visualization

In [ ]:
if sqstm1_idx is not None:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot 1: Box plot by tau status
    tau_pos_expr = sqstm1_df[sqstm1_df['tau_positive'] == 1]['expression']
    tau_neg_expr = sqstm1_df[sqstm1_df['tau_positive'] == 0]['expression']
    
    bp = axes[0, 0].boxplot([tau_neg_expr, tau_pos_expr], 
                            labels=['Tau-', 'Tau+'],
                            patch_artist=True)
    bp['boxes'][0].set_facecolor('blue')
    bp['boxes'][1].set_facecolor('red')
    axes[0, 0].set_ylabel('SQSTM1 Expression (log2)', fontsize=12)
    axes[0, 0].set_title('SQSTM1 Expression by Tau Status', fontsize=14, fontweight='bold')
    
    # Add fold change annotation
    if 'log2FoldChange' in results_sqstm1.columns:
        fc = results_sqstm1.iloc[0]['log2FoldChange']
        pval = results_sqstm1.iloc[0]['pvalue'] if 'pvalue' in results_sqstm1.columns else None
        axes[0, 0].text(0.5, 0.95, f'Log2FC = {fc:.2f}\np = {pval:.2e}' if pval else f'Log2FC = {fc:.2f}',
                       transform=axes[0, 0].transAxes, ha='center', va='top',
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Plot 2: Violin plot
    parts = axes[0, 1].violinplot([tau_neg_expr, tau_pos_expr], positions=[0, 1], 
                                  showmeans=True, showmedians=True)
    axes[0, 1].set_xticks([0, 1])
    axes[0, 1].set_xticklabels(['Tau-', 'Tau+'])
    axes[0, 1].set_ylabel('SQSTM1 Expression (log2)', fontsize=12)
    axes[0, 1].set_title('Distribution Comparison', fontsize=14, fontweight='bold')
    
    # Plot 3: Pseudotime correlation
    if 'pseudotime' in sqstm1_df.columns and not sqstm1_df['pseudotime'].isna().all():
        # Color by tau status
        colors = ['red' if x == 1 else 'blue' for x in sqstm1_df['tau_positive']]
        axes[1, 0].scatter(sqstm1_df['pseudotime'], sqstm1_df['expression'],
                          c=colors, alpha=0.6, s=30)
        
        # Add regression line if available
        if beta is not None:
            x_line = np.linspace(sqstm1_df['pseudotime'].min(), 
                                sqstm1_df['pseudotime'].max(), 100)
            y_line = lr.intercept_ + beta * x_line
            axes[1, 0].plot(x_line, y_line, 'k--', alpha=0.8, 
                           label=f'β = {beta:.3f}, R² = {r_squared:.3f}')
            axes[1, 0].legend()
        
        axes[1, 0].set_xlabel('Pseudotime', fontsize=12)
        axes[1, 0].set_ylabel('SQSTM1 Expression (log2)', fontsize=12)
        axes[1, 0].set_title('SQSTM1 vs Disease Progression', fontsize=14, fontweight='bold')
    else:
        axes[1, 0].text(0.5, 0.5, 'Pseudotime not available',
                       ha='center', va='center', transform=axes[1, 0].transAxes)
    
    # Plot 4: Effect size comparison
    if 'log2FoldChange' in results_sqstm1.columns:
        observed_fc = results_sqstm1.iloc[0]['log2FoldChange']
        claimed_fc = 3.413
        
        bars = axes[1, 1].bar(['Claimed', 'Observed'], [claimed_fc, observed_fc],
                             color=['gray', 'green'], alpha=0.7)
        axes[1, 1].set_ylabel('Log2 Fold Change', fontsize=12)
        axes[1, 1].set_title('Fold Change Comparison', fontsize=14, fontweight='bold')
        axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        
        # Add value labels
        for bar, val in zip(bars, [claimed_fc, observed_fc]):
            axes[1, 1].text(bar.get_x() + bar.get_width()/2, val + 0.1,
                           f'{val:.2f}', ha='center', va='bottom', fontweight='bold')
    
    plt.suptitle('SQSTM1/p62 Comprehensive Analysis', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('sqstm1_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

## 6. Evaluate Claim

In [ ]:
print("\n" + "="*60)
print("CLAIM EVALUATION")
print("="*60)
print("Claim: SQSTM1 is massively upregulated")
print("       log2FC = 3.413, FDR = 1.76 × 10^-8")
print("       Increases with pseudotime (β = 4.951, FDR < 0.001)")
print()

if sqstm1_idx is not None and 'log2FoldChange' in results_sqstm1.columns:
    # Part 1: Fold change evaluation
    observed_fc = results_sqstm1.iloc[0]['log2FoldChange']
    claimed_fc = 3.413
    fc_diff_pct = abs(observed_fc - claimed_fc) / claimed_fc * 100
    
    print("Part 1: Fold Change")
    print(f"  Claimed: {claimed_fc:.3f}")
    print(f"  Observed: {observed_fc:.3f}")
    print(f"  Difference: {fc_diff_pct:.1f}%")
    
    if fc_diff_pct < 20:
        fc_verdict = "SUPPORTED"
    elif fc_diff_pct < 50:
        fc_verdict = "PARTIALLY SUPPORTED"
    else:
        fc_verdict = "REFUTED"
    print(f"  Verdict: {fc_verdict}")
    
    # Part 2: Significance evaluation
    print("\nPart 2: Statistical Significance")
    if 'pvalue' in results_sqstm1.columns:
        observed_p = results_sqstm1.iloc[0]['pvalue']
        claimed_p = 1.76e-8
        
        print(f"  Claimed FDR: {claimed_p:.2e}")
        print(f"  Observed p-value: {observed_p:.2e}")
        
        if observed_p < 0.001:
            sig_verdict = "SUPPORTED"
        elif observed_p < 0.05:
            sig_verdict = "PARTIALLY SUPPORTED"
        else:
            sig_verdict = "REFUTED"
        print(f"  Verdict: {sig_verdict}")
    
    # Part 3: Pseudotime correlation
    if beta is not None:
        print("\nPart 3: Pseudotime Correlation")
        claimed_beta = 4.951
        beta_diff_pct = abs(beta - claimed_beta) / claimed_beta * 100 if claimed_beta != 0 else 100
        
        print(f"  Claimed β: {claimed_beta:.3f}")
        print(f"  Observed β: {beta:.3f}")
        print(f"  Difference: {beta_diff_pct:.1f}%")
        
        if beta_diff_pct < 30 and beta > 0:
            time_verdict = "SUPPORTED"
        elif beta > 0:
            time_verdict = "PARTIALLY SUPPORTED"
        else:
            time_verdict = "REFUTED"
        print(f"  Verdict: {time_verdict}")
    
    # Overall verdict
    print("\n" + "="*60)
    print("OVERALL VERDICT:")
    
    # Check if SQSTM1 is upregulated
    is_upregulated = observed_fc > 0.5  # At least moderate upregulation
    is_significant = 'pvalue' in results_sqstm1.columns and results_sqstm1.iloc[0]['pvalue'] < 0.05
    
    if is_upregulated and is_significant:
        if fc_diff_pct < 30:
            overall = "SUPPORTED - SQSTM1 is significantly upregulated as claimed"
        else:
            overall = f"PARTIALLY SUPPORTED - SQSTM1 is upregulated but fold change differs ({observed_fc:.2f} vs {claimed_fc:.2f})"
    elif is_upregulated:
        overall = "PARTIALLY SUPPORTED - SQSTM1 is upregulated but not as strongly as claimed"
    else:
        overall = "REFUTED - SQSTM1 is not significantly upregulated"
    
    print(overall)
else:
    print("ERROR: Could not evaluate claim - SQSTM1 data not available")

## 7. Save Results

In [ ]:
if sqstm1_idx is not None:
    # Compile comprehensive results
    comprehensive_results = {
        'protein': 'SQSTM1',
        'claimed_log2FC': 3.413,
        'observed_log2FC': results_sqstm1.iloc[0]['log2FoldChange'] if 'log2FoldChange' in results_sqstm1.columns else None,
        'claimed_FDR': 1.76e-8,
        'observed_pvalue': results_sqstm1.iloc[0]['pvalue'] if 'pvalue' in results_sqstm1.columns else None,
        'claimed_beta': 4.951,
        'observed_beta': beta,
        'r_squared': r_squared,
        'n_tau_positive': (sqstm1_df['tau_positive'] == 1).sum(),
        'n_tau_negative': (sqstm1_df['tau_positive'] == 0).sum(),
        'mean_expr_tau_pos': sqstm1_df[sqstm1_df['tau_positive'] == 1]['expression'].mean(),
        'mean_expr_tau_neg': sqstm1_df[sqstm1_df['tau_positive'] == 0]['expression'].mean(),
        'verdict': overall if 'overall' in locals() else 'Not evaluated'
    }
    
    # Save to CSV
    output_df = pd.DataFrame([comprehensive_results])
    output_df.to_csv('../05_statistical_reports/claim2_sqstm1_results.csv', index=False)
    print("\nResults saved to: ../05_statistical_reports/claim2_sqstm1_results.csv")
    
    # Display summary
    print("\nFinal Summary:")
    for key, value in comprehensive_results.items():
        if value is not None:
            if isinstance(value, float):
                print(f"  {key}: {value:.3f}" if value > 0.01 else f"  {key}: {value:.3e}")
            else:
                print(f"  {key}: {value}")

## Summary

This PertPy-based analysis of SQSTM1/p62:

1. **Tests differential expression** between tau-positive and tau-negative neurons
2. **Evaluates the claim** of massive upregulation (log2FC = 3.413)
3. **Analyzes pseudotime correlation** to assess disease progression relationship
4. **Provides comprehensive visualization** of expression patterns
5. **Delivers objective verdict** based on observed vs claimed values

The analysis determines whether SQSTM1 upregulation is as dramatic as claimed and whether it correlates with disease progression.